# Project 4: Open-ended
---

This notebook is supposed to be used to provide the solution to the project 4 of the module Introduction to Machine Learning 2019 @ ETHZ.

---


## Environmental Set-Up

We first set the environment and load the later required packages, as well as fix the random seed globally.

In [0]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sn
import sklearn as sl
import datetime
import random
import matplotlib.pyplot as plt
import time
import copy

random_seed = 2208

%matplotlib inline
sn.set_context('notebook')
%config InlineBackend.figure_format = 'retina'
random.seed(random_seed)
warnings.filterwarnings('ignore')

After loading the basic packages, we will now install Pytorch on the virtual machine since we gonna use it to apply neural networks to solve the project as suggested. Pytorch is chosen as it provides a according to the subjective opinion of the author nice interface compared to Tensorflow, but speedwise supposingly outperforms Keras.

In [0]:
!pip3 install pandas==0.24.2
!pip3 install torch

Since the Google Colab platform offers us a GPU, we will make sure to tell pytorch to use it, as it will speed up the training of our neural network significantly. Unfortunately up until now Pytorch does not support the use of TPUs (Google Colab would offer those as well).

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
import torch.utils.data as data_utils
use_cuda = True

torch.manual_seed(2208)

---

## Load in the data

We now use the Google Colab API to load the data and the sample submission from disk into the temproray cloud storage attached to this PaaS (platform as a service) solution to make it accessible.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Alternatively one can connect Colab with Google Drive for larger data transmission rates.

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


---
## Project 4

The following section now solves the project 4 of the Introduction to Machine Learning course 2019.

---

### Formatting the data

Although the data is loaded we format it to have it in the handy pandas data frame format.

In [0]:
# Get the labeled train data
train_labeled = pd.read_hdf("train_labeled.h5", "train")
train_labeled.describe()



In [0]:
# Get the unlabeled train data
train_unlabeled = pd.read_hdf("train_unlabeled.h5", "train")
train_unlabeled.head()



In [0]:
# Get the test data
test = pd.read_hdf("test.h5", "test")
test.head()

We quickly inspect the shape of the data to make sure the data has been correctly loaded and casted into a pandas data frame.

In [0]:
print("train labeled shape: ", np.array(train_labeled).shape)
print("train unlabeled shape: ", np.array(train_unlabeled).shape)
print("test shape: ", np.array(test).shape)

That looks very good. We seperate the label from the features for the sake of handiness of our implementations and data handling in the following.

In [0]:
X_train_labeled = train_labeled.iloc[:, 1:]
y_train_labeled = train_labeled.iloc[:, 0]

In [0]:
'''
Get sample prediction file format.
Sample predictions will be simply replaced with the ones obtained from the
custom model.
''' 



In [0]:
submission = pd.read_csv('sample.csv', index_col=0, float_precision='high')
submission.head()

---

### Exploratory Data Analysis

Before starting with trying to model the data, we will have a first look at the data. First we will look at the distribution of the labels in the training data, since knowing if the data is balanced or not heavily influences the choice of algorithms we will consider later on.

In [0]:
n, bins, patches = plt.hist(np.array(y_train_labeled),
                            facecolor='b', alpha=0.75, align="mid")


plt.xlabel('Class Label')
plt.ylabel('Rel. Frequency')
plt.title('Histogram of Class Labels in the Training Data')
plt.axis([0, 9, 0, 2000])
plt.grid(True)
plt.show()

In [0]:
from sklearn.utils.extmath import randomized_svd
from sklearn.preprocessing import StandardScaler
X_train_std = StandardScaler().fit_transform(X_train_labeled)
U,s,Vt = randomized_svd(np.array(X_train_std), n_components=139)



In [0]:
plt.plot(s**2)
plt.ylabel('Eigenvalue spectrum')
plt.show()

We see that the number of training samples we have for each class is about the same. So problems arising from class imbalance seem to be not likely for the given task and we will not consider special approaches oriented towards adressing this.

Although it might be not that informative given the relative high number of features, let us quickly inspect the correlation structure of the features.

In [0]:
corr = X_train_labeled.corr()
print(X_train_labeled.shape)

f, ax = plt.subplots(figsize=(10, 8))
ax.set_title("Heatmap of the correlation structure")
sn.heatmap(
    corr,
    mask=np.zeros_like(corr, dtype=np.bool),
    cmap=sn.diverging_palette(220, 10, as_cmap=True),
    square=True,
    ax=ax)
plt.subplots_adjust(bottom=0.25)
plt.show()

At first sight it seems that we have rather strong correlations but at least no 1-to-1 mappings of the individual features. We will confirm this by looking at the 10 feature-pairs that are the most correlated.

In [0]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [0]:
print("Top Absolute Correlations")
print(get_top_abs_correlations(X_train_labeled, 10))

As indicated by our heat map we see tthat we have no correlation of 1 and hence for now no reason to exclude any features from the beginning.



---

### More Sophisticated Experiments: 4-Layer ANN

After trying out a number of different approaches, including combinations of dimensionality reduction and classification approachs like NNMF and SVMs, many semi-supervised approaches such as Ladder networks and beta-VAEs, we will go one step back and will perform sophisticated trials a NN fitted to the labeled data only. to get a better performance.
To this end we use methods from the following resource: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#model-training-and-validation-code .

The actual procedure is explained in the following.

In [0]:
# check if GPU is available and set the device accordingly
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
torch.cuda.get_device_name(0)

1. We will split our loaded data into a training and validation set. The former will be used for training purposes, while the latter will be used to monitor the performance of our network. We have seen several times that performance estimate we get on the training set will be too optimistic and should thus in favor of the estimate based on the validation set not used for model tuning purposes. The sci-kit learn library provides all we need to realize the split. We then transform the data in such a way that it fits in the pytorch framework.

In [0]:
# Create train val split and create the data loader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(np.concatenate((np.array(X_train_labeled), np.array(train_unlabeled))))
X_train_standardized = sc.transform(X_train_labeled)

data_train, data_val, label_train, label_val = train_test_split(X_train_standardized, 
                                                                y_train_labeled, test_size = 0.15, 
                                                                random_state=2208)
print(np.array(data_train).shape)
print(np.array(label_train).shape)
print(np.array(data_val).shape)
print(np.array(label_val).shape)

# Note that from here on we expect GPU to be available, if that is not the case 
# use torch.xxxTensor instead of torch.cuda.xxxTensor

train_tensors = data_utils.TensorDataset(
    torch.cuda.FloatTensor(np.array(data_train)), 
    torch.cuda.LongTensor(np.array(label_train)))

train_loader = data_utils.DataLoader(train_tensors, 
                                     batch_size = 32, shuffle = True)

val_tensors = data_utils.TensorDataset(
    torch.cuda.FloatTensor(np.array(data_val)), 
    torch.cuda.LongTensor(np.array(label_val)))

val_loader = data_utils.DataLoader(val_tensors, 
                                   batch_size = 32, shuffle = True)

data_loaders_dict = {'train':train_loader, 'val':val_loader}
data_loaders_dict

2. Now we define the function to train a preset torch.nn model and thereby monitor the performance of it. This is again inspired by the previously referenced official pytorch tutorial. The nice thing about this function is that it will allow us to monitor the model performance on both the training and validation set at each epoch and will return the best found model i.e. the one with the highest validation accuracy.

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=100):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.type(torch.FloatTensor).to(device)
                labels = labels.type(torch.LongTensor).to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                  # Get model outputs and calculate loss
                  outputs = model(inputs)
                  loss = criterion(outputs, labels)
                  _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                if phase == 'train':
                  loss.backward()
                  optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.6f} Acc: {:.6f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model if it has the best val accurary
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, 
                                                        time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

3. We now set up the model i.e. the network structure. We will use a very basic model that consists only of two hidden layers, but sufficiently many neurons in each of those to resemble different transformation of the input features. We also write a quick function to initialize our weights with a xavier_uniform distribution as suggested in recent literature.

In [0]:
def init_weights(m):
  if type(m) == nn.Linear:
    torch.nn.init.xavier_uniform(m.weight)
    m.bias.data.fill_(0.01)

In [0]:
seqnet = nn.Sequential(nn.Linear(139, 512), nn.BatchNorm1d(512), nn.LeakyReLU(), nn.Dropout(0.6), 
                       nn.Linear(512, 256), nn.BatchNorm1d(256), nn.LeakyReLU(), nn.Dropout(0.5),
                       nn.Linear(256, 256), nn.BatchNorm1d(256), nn.LeakyReLU(), nn.Dropout(0.5),
                       nn.Linear(256, 256), nn.BatchNorm1d(256), nn.LeakyReLU(), nn.Dropout(0.4),
                      nn.Linear(256, 10))
seqnet.apply(init_weights)
seqnet.cuda()
seqnet.to(device)
print(seqnet)

4. We now set up the optimizer and the criterion. We thereby use the AdamW optimizer as it automatically adapts the learning rate and has been shown to perform superior to the Adam optimizer. We use an existing implementation as given in the following : https://github.com/egg-west/AdamW-pytorch
So far our output layer consists of 10 neurons outputting values in $\mathbb{R}$, we will use the CrossEntropyLoss criterion provided by pytorch. Additionally we apply batch normalization as it has been shown that this improves the speed of convergence of the network. We also used dropout layers in our network structure to prevent overfitting and make our model more robust against noise.

In [0]:
import math
import torch
from torch.optim.optimizer import Optimizer

class AdamW(Optimizer):
    """Implements Adam algorithm.
    It has been proposed in `Adam: A Method for Stochastic Optimization`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        amsgrad (boolean, optional): whether to use the AMSGrad variant of this
            algorithm from the paper `On the Convergence of Adam and Beyond`_
    .. _Adam\: A Method for Stochastic Optimization:
        https://arxiv.org/abs/1412.6980
    .. _On the Convergence of Adam and Beyond:
        https://openreview.net/forum?id=ryQu7f-RZ
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # if group['weight_decay'] != 0:
                #     grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                if amsgrad:
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # Use the max. for normalizing running avg. of gradient
                    denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                else:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                # p.data.addcdiv_(-step_size, exp_avg, denom)
                p.data.add_(-step_size,  torch.mul(p.data, group['weight_decay']).addcdiv_(1, exp_avg, denom) )

        return loss

We set the learning rate to 1e-5 to ensure that the step size is small enough to let the network converge.

In [0]:
params_to_update = seqnet.parameters()
optimizer_ft = AdamW(params_to_update, lr=1e-5)
criterion = nn.CrossEntropyLoss(size_average=True)

Before we start training our model, let us quickly as a last control mechanism check if the split of the train and validation split was done such that the distribution of the class labels is roughly the same in both.

In [0]:
counts_train = np.unique(label_train, return_counts=True)[1]
counts_val = np.unique(label_val, return_counts=True)[1]
(counts_train, counts_val)

That is the case.

5. Having done all of this we are good to go and can train our model.

In [0]:
num_epochs=500
seqnet_fit, hist = train_model(seqnet, data_loaders_dict, 
                             criterion, optimizer_ft, 
                             num_epochs=num_epochs)

6. The validation accuracy looks promosing. However to get an idea of how representative that score is for the performance on the test set we will now predict the values for the test set and create a submission from those predictions.

In [0]:
new_test_data = ae.encoder(torch.cuda.FloatTensor(np.array(test))).detach().cpu().numpy()
new_test_data

In [0]:
#x_test = np.concatenate((np.array(test), new_test_data), axis=1)
test_standardized = sc.transform(np.array(test))
test_set = torch.from_numpy(np.array(test_standardized))
predictions = []

for value in test_set:
  # then put it on the GPU, make it float and insert a fake batch dimension
  test_value = Variable(value.cuda())
  test_value = test_value.float()
  test_value = test_value.unsqueeze(0)

  # pass it through the model
  outputs = seqnet_fit(test_value)
  _, preds = torch.max(outputs, 1)

# get the result out and reshape it
  predictions.append(preds.cpu().numpy()[0])

predictions = np.array(predictions)
predictions

Let us quickly check the fitted distribution of the class labels.

In [0]:
unique, counts = np.unique(predictions, return_counts=True)
dict(zip(unique, counts))

Well that looks quite nice, recalling the distribution of the labels of our training and validation data. As we assume that the data for which we predicted the labels comes from the same distribution as the one which generated our training and validation data, this is exactly what we would expect.

Let us now finally create a submission based on those predictions and download it, such that we potentially could hand it in.

---
### Create submission

In [0]:
submission['y'] = predictions
submission.head()

---

## Export data

We finally use the Google Colab API to download our submission data frame in from of an csv, that we can submit to the submission platform.

In [0]:
from google.colab import files

ts = str(datetime.datetime.utcnow())
ts = ts.replace(' ', '_')
Filename = 'submission_name' #@param {type:"string"}
fname = Filename+ts+'.csv'

with open(fname, 'w') as f:
  submission.to_csv(f, float_format='%.64f', index=True, header=True)

files.download(fname)